This file analyzes the role of cell-cell contacts, modeled as a positionally dependent on rate

First load the packages

In [ ]:
from oneD_simulation import simulation, phase_space
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
plt.rcParams.update({'pdf.fonttype': 42})


Simulation dynamics
------

Perform two simulations ("sim" and "sim2"), one without cell-cell contacts and one with. 

Simulating with cell-cell contacts involves a positionally biased association term, coded in the "solve" function with the parameter "apical_on = True"

Details are provided in the Supplementary Modeling

Plots are saved to the "plots" directory 

In [ ]:
sim = simulation()
sim.set_num_x(100,apical_on=True)
sim.set_t_span(10,1000)
sim.set_initial(40,1)
sim.k_on = sim.k_off
sim.solve()
sim.get_apical_solution()
N = 10
cmap = cm.Reds(np.arange(N)/N)
sim.plot_time_series(cmap,show=True,apical=True,ylim=(0,200))


sim2 = simulation()
sim2.set_num_x(100,apical_on=True)
sim2.set_t_span(10,1000)
sim2.set_initial(40,1,apical_on=True)
sim2.solve(apical_on=True)
sim2.get_apical_solution()
N = 10
cmap = cm.Reds(np.arange(N)/N)
sim2.plot_time_series(cmap,show=True,apical=True,ylim=(0,200))


Plot kymographs of simulations with and without cell-cell contacts

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(4,4))
fig.subplots_adjust(wspace=0.15, hspace=None)
ax[0].set_title("Without \n cell-contacts",fontsize=12)
ax[1].set_title("With \n cell-contacts",fontsize=12)

vmin=10;vmax=100

t_span = sim.t_span

extent = [0,100,0,sim.l_apical]
ax[0].imshow(sim.ysol_apical.T,extent=extent,vmin=vmin,vmax=vmax)
ax[1].imshow(sim2.ysol_apical.T,extent = extent,vmin=vmin,vmax=vmax)
ax[0].set(aspect=(extent[0]-extent[1])/(extent[2]-extent[3]),xlabel="Time (%)",ylabel="Position (µm)")
ax[1].set(aspect=(extent[0]-extent[1])/(extent[2]-extent[3]),xlabel="Time (%)")
ax[1].axes.set_yticks([])
fig.savefig("plots/cell_contacts kymo.pdf",dpi=300)

Orientation bias when in the presence of cell-cell contacts
---

Some functions to calculate the orientation of the apical domain. And to generate the rose plots

In [ ]:
def get_angles(n,apical_on):
    theta = []
    for i in range(n):
        sim2 = simulation()
        sim2.set_num_x(100, apical_on=True)
        sim2.set_t_span(10, 900)
        sim2.set_initial(40, 1)
        sim2.solve(apical_on=apical_on)
        i_max = np.absolute(sim2.ysol[-1] - sim2.ysol[-1].max()).argmin()
        theta.append(2*np.pi*i_max/sim2.num_x)
    return theta



def rose_plot(ax, angles, bins=16, density=None, offset=0, lab_unit="degrees",
              start_zero=False, **param_dict):
    """
    Plot polar histogram of angles on ax. ax must have been created using
    subplot_kw=dict(projection='polar'). Angles are expected in radians.
    
    Courtesy of https://jwalton.info/Matplotlib-rose-plots/
    """
    # Wrap angles to [-pi, pi)
    angles = (angles + np.pi) % (2*np.pi) - np.pi

    # Set bins symetrically around zero
    if start_zero:
        # To have a bin edge at zero use an even number of bins
        if bins % 2:
            bins += 1
        bins = np.linspace(-np.pi, np.pi, num=bins+1)

    # Bin data and record counts
    count, bin = np.histogram(angles, bins=bins)

    # Compute width of each bin
    widths = np.diff(bin) + 1e-2

    # By default plot density (frequency potentially misleading)
    if density is None or density is True:
        # Area to assign each bin
        area = count / angles.size
        # Calculate corresponding bin radius
        radius = (area / np.pi)**.5
    else:
        radius = count

    # Plot data on ax
    ax.bar(bin[:-1], radius, zorder=1, align='edge', width=widths,
            color="darkred",linewidth=0)

    # Set the direction of the zero angle
    ax.set_theta_offset(offset)

    # Remove ylabels, they are mostly obstructive and not informative
    ax.set_yticks([])

    if lab_unit == "radians":
        label = [r'$\pi/2$', r'$\pi/4$','$0$', r'$-\pi/4$', r'$-\pi/2$', r'$-3\pi/4$',
                  r'$\pi$', r'$-3\pi/4$']
        ax.set_xticklabels(label)
    if lab_unit == "degrees":
        label = [r'$90 \degree$', r'$45 \degree$', r'$0 \degree$', r'$-45 \degree$', r'$-90 \degree$', r'$-135 \degree$',
                 r'$180 \degree$', r'$135 \degree$']
        ax.set_xticklabels(label)
    ax.yaxis.grid(False)

Perform the simulations. Each condition is simulated 100 times, given different random initial conditions

In [ ]:
wcc = get_angles(n=100,apical_on=True)
wocc = get_angles(n=100,apical_on=False)
angles0 = -np.array(wocc) - np.pi/2
angles1 = -np.array(wcc) - np.pi/2

Plot the results

In [ ]:
fig, ax = plt.subplots(1, 2, subplot_kw=dict(projection='polar'),figsize=(5,3))
rose_plot(ax[0], angles0,start_zero=True,bins=21,density=True, lab_unit="degrees",offset=0)
rose_plot(ax[1], angles1,bins=21,start_zero=True, density=True,lab_unit="degrees",offset=0)
for axx in ax:
    axx.set(ylim=(0,0.6))
    axx.grid(b=None)
# fig.GridColor = 'red'
ax[0].set_title("Without \n cell-cell contacts \n")
ax[1].set_title("With \n cell-cell contacts \n")
fig.show()
fig.savefig("plots/rose.pdf",dpi=300)